# Lending Loop Web App
This is a prototype for a Lending Loop web app / dashboard. This notebook focuses on the logics of Lending Loop summary calculations.

# System Initializations

### Package Initializations

In [1]:
# DataFrames
import pandas as pd
import findspark
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T

# Numerical Packages
import numpy as np
import math

# Datetime
import datetime

# Misc Packages
import os

# Visualizations
import plotly
import plotly.offline as pyo
import plotly.graph_objs as go

### System Configuration


In [2]:
# Plotly offline
pyo.init_notebook_mode(connected=True)

# Pandas display
pd.set_option('display.max_colwidth', -1)

# PySpark Session Initialization
packages = 'org.mongodb.spark:mongo-spark-connector_2.11:2.2.0'
dedicated_memory = '1g'

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages {} --driver-memory {} pyspark-shell' \
    .format(packages, dedicated_memory)

In [3]:
# Find SPARK_HOME
findspark.init()

# Create SparkSession
spark = (pyspark.sql.SparkSession
         .builder.appName('LendingLoop')
         .getOrCreate())

### Constant Definitions

In [4]:
LENDING_LOOP_FEE_RATE = 1.5

### Helper Function Definitions

In [5]:
def display_DF(sparkDF, n=5):
    '''
    Interactively displays the first n rows of a sparkDF as a pandas dataframe
    '''
    return (sparkDF
            .limit(n)
            .drop('_id')
            .toPandas())

def import_new_notes(DF):
    '''
    Search DF for new notes and import them into MongoDB.
    '''
    @F.udf(returnType=T.DoubleType())
    def calculate_principal(unitPay, interestRate, totalPayCycles):
        '''
        Calculate the principal invested based on the unitPay, interestRate and totalPayCycles.
        '''
        interestRate /= 100 * 12

        return unitPay / interestRate * (1 - 1. / (1 + interestRate)**totalPayCycles)
    
    @F.udf(returnType=T.FloatType())
    def bad_debt_funds(principal, grade, cyclesTotal):
        rate = interestRatesBroadcast.value[grade] / 1200
        return round(principal * (cyclesTotal * rate / (1 - (1 + rate)**(-cyclesTotal)) - 1), 2)
    
    # Obtain list of new notes from input DF
    newNotesDF = (DF
                  .orderBy('dueDate')
                  .groupBy('loanID', 'company', 'loanName', 'interestRate', 'grade')
                  .agg(F.count('loanID').alias('cyclesTotal'), 
                       F.sum('principalScheduled').alias('principal'),
                       F.round(F.sum('fees'), 2).alias('fees'),
                       F.round(F.sum('interestScheduled') - F.sum('fees'), 2).alias('profits'),
                       F.round(F.mean('totalScheduled'), 2).alias('unitPayment'), 
                       F.add_months(F.first('dueDate'), -1).alias('startDate'))
                  .withColumn('principal', 
                              F.round(calculate_principal('unitPayment', 'interestRate', 'cyclesTotal'), 0))
                  .withColumn('cyclesRemaining', F.udf(lambda x: x, T.LongType())('cyclesTotal'))
                  .withColumn('badDebtFunds', bad_debt_funds('principal', 'grade', 'cyclesTotal'))
                  .withColumn('profits', F.col('profits') - F.col('badDebtFunds'))
                  .withColumn('amountRepayed', F.lit(0.00))
                  .cache())
    
    return newNotesDF

# Test Area

### Import dataset

In [6]:
# Read raw CSV file
rawDF = (spark
         .read
         .format('com.databricks.spark.csv')
         .option('header', 'True')
         .option('inferschema', 'True')
         .load('all_payments.csv'))

# Camel case titles
camelCaseDict = {title: title[0].lower() + title.replace(' ', '')[1:] for title in rawDF.columns}

# Simplify certain column titles
camelCaseDict['Fees Paid to Loop'] = 'fees'
camelCaseDict['Risk Band'] = 'grade'
camelCaseDict['Loan Id'] = 'loanID'

# Camelcase column titles
rawDF = rawDF.select([F.col(title).alias(camelCaseDict[title]) for title in camelCaseDict.keys()])

In [7]:
display_DF(rawDF)

,status,interestPaid,paymentType,principalScheduled,totalOwed,loanID,grade,interestRate,company,principalPaid,dueDate,interestScheduled,totalScheduled,fees,loanName,principalOwed,totalPaid,interestOwed,datePaid
0,Paid,0.64,Scheduled Payment,2.83,3.47,83910,B+,10.33,Bronze Baxx Tanning Studio Inc.,2.83,2017-09-22,0.64,3.47,0.09,Salon - Equipment Purchase (1 of 2),2.83,3.47,0.64,2017-09-22
1,Paid,0.86,Scheduled Payment,2.39,3.25,59705,D,20.67,Nice and Smooth Ultramedia Inc.,2.39,2017-09-22,0.86,3.25,0.06,Concert Producer & Promoter,2.39,3.25,0.86,2017-09-23
2,Paid,0.64,Scheduled Payment,1.27,1.91,7802,B+,10.33,Brightpath Capital Corporation,1.27,2017-09-25,0.64,1.91,0.09,Mortgage Lender - Working Capital,1.27,1.91,0.64,2017-09-25
3,Scheduled,0.00,Scheduled Payment,0.82,1.83,96625,C,16.18,Rossco's Tree Service and contracting Ltd.,0.00,2017-09-30,1.01,1.83,0.09,Arborist - Lease Buyouts (1 of 3),0.82,0.00,1.01,NaT
4,Scheduled,0.00,Scheduled Payment,0.53,0.91,36403,D+,18.40,Social Theorem Consulting Inc,0.00,2017-10-07,0.38,0.91,0.03,Digital Media Agency - Refinancing And Continued Expansion,0.53,0.00,0.38,NaT


### Initialise Base `Summary` State

In [8]:
# # TODO Update to realistic values
# baseState = {
#     'lifeTimeEarnings': 0.,
#     'netROI': 0.,
#     'availableFunds': 1500,
#     'fundsInvested': 0.,
#     'fundCommited': 0.,
#     'lastUpdated': datetime.datetime(2017,9,1),
# }

### Add new notes to collection

In [9]:
# import_new_notes(rawDF)

### Define Grades interest Rate

In [10]:
interestRates = {
    'A+': 0.56,
    'A': 1.83,
    'B+': 3.05,
    'B': 4.32,
    'C+': 5.46,
    'C': 6.25,
    'D+': 7.11,
    'D': 8.00,
    'E+': 8.84,
    'E': 9.64,
}

# Broadcast interest rates
interestRatesBroadcast = spark.sparkContext.broadcast(interestRates)

### Pull new collection

In [11]:
notesDF = import_new_notes(rawDF)
display_DF(notesDF)

,loanID,company,loanName,interestRate,grade,cyclesTotal,principal,fees,profits,unitPayment,startDate,cyclesRemaining,badDebtFunds,amountRepayed
0,7802,Brightpath Capital Corporation,Mortgage Lender - Working Capital,10.33,B+,48,75.0,2.45,9.44,1.91,2017-08-25,48,4.76,0.0
1,83910,Bronze Baxx Tanning Studio Inc.,Salon - Equipment Purchase (1 of 2),10.33,B+,24,75.0,1.21,4.59,3.47,2017-08-22,24,2.41,0.0
2,90873,Nitin Chauhan Medicine Professional Corporation,"Head, Neck & Facial Plastic Surgeon - Working Capital",8.52,A,24,50.0,0.81,2.66,2.27,2017-09-07,24,0.96,0.0
3,67770,Jenco Canada Inc.,LED Lighting Distributor and Franchisor,8.52,A,12,50.0,0.42,1.38,4.36,2017-09-07,12,0.50,0.0
4,96625,Rossco's Tree Service and contracting Ltd.,Arborist - Lease Buyouts (1 of 3),16.18,C,60,75.0,3.23,18.83,1.83,2017-08-31,60,12.52,0.0


### Import New Transactions

In [12]:
def update_state(rawDF, notesDF):
    '''
    Update state with new transactions from DF.
    '''
    # Obtain transactions that have occured
    netTransactionsDF = (rawDF
                         .filter(F.isnull('datePaid') == 'False')
                         .groupBy('loanID')
                         .agg(F.sum('totalPaid').alias('totalPaid')))
    
    # Update noteStates
    updatedNotesStateDF = (notesDF
                           .join(netTransactionsDF, 'loanID', 'left_outer')
                           .withColumn('amountRepayed', F.col('amountRepayed') + F.col('totalPaid'))
                           .withColumn('cyclesRemaining', F.col('cyclesRemaining') - 1)
                           .drop('totalPaid')
                           .fillna(0.0)
                           .cache())
    
    return updatedNotesStateDF

In [13]:
notesDF = update_state(rawDF, notesDF)
display_DF(notesDF, 10)

,loanID,company,loanName,interestRate,grade,cyclesTotal,principal,fees,profits,unitPayment,startDate,cyclesRemaining,badDebtFunds,amountRepayed
0,7802,Brightpath Capital Corporation,Mortgage Lender - Working Capital,10.33,B+,48,75.0,2.45,9.44,1.91,2017-08-25,47,4.76,1.91
1,83910,Bronze Baxx Tanning Studio Inc.,Salon - Equipment Purchase (1 of 2),10.33,B+,24,75.0,1.21,4.59,3.47,2017-08-22,23,2.41,3.47
2,90873,Nitin Chauhan Medicine Professional Corporation,"Head, Neck & Facial Plastic Surgeon - Working Capital",8.52,A,24,50.0,0.81,2.66,2.27,2017-09-07,23,0.96,0.00
3,67770,Jenco Canada Inc.,LED Lighting Distributor and Franchisor,8.52,A,12,50.0,0.42,1.38,4.36,2017-09-07,11,0.50,0.00
4,96625,Rossco's Tree Service and contracting Ltd.,Arborist - Lease Buyouts (1 of 3),16.18,C,60,75.0,3.23,18.83,1.83,2017-08-31,59,12.52,0.00
5,83061,J.K. ENGINEERING LTD,Infrastructure Engineering and Construction Firm - Second Loan,8.52,A,36,100.0,2.42,8.21,3.15,2017-09-15,35,2.85,0.00
6,73245,IT MULTISERVICES MG&CG INC.,E-commerce Company - Inventory Purchase,16.18,C,24,25.0,0.41,2.22,1.22,2017-09-07,23,1.66,0.00
7,36403,Social Theorem Consulting Inc,Digital Media Agency - Refinancing And Continued Expansion,18.40,D+,36,25.0,0.63,4.06,0.90,2017-09-07,35,2.83,0.00
8,45851,The Extensionist Consultancy Inc,Salon Chain - Inventory Purchase,12.20,B,36,75.0,1.84,7.80,2.49,2017-09-14,35,5.10,0.00
9,87799,1005351 BC LTD. O/A Liberty Tax Service,Liberty Tax - Territory Expansion,10.33,B+,48,25.0,0.82,2.99,0.63,2017-09-24,47,1.59,0.00


### Create custom progress bar

In [14]:
def progress_bar(DF):
    '''
    Creates a custom progress bar based on notesDF.
    '''
    def _convert_to_list(colName):
        '''
        Extract the colName of notesDF as a list, with each entry rounded to 2 decimal places.
        '''
        return (DF
                .select(colName)
                .rdd
                .map(lambda x: x[0] if colName == 'company' or colName == 'monthsToBreakeven' else round(x[0], 2))
                .collect())
    
    def _create_trace(colName):
        '''
        Create a trace based on colName.
        '''
        colourDict = {
            'feesPaid': '#585858', 
            'nextFeesPayment': '#848484', 
            'feesOutstanding': '#BDBDBD', 
            'principalReceived': '#512361',
            'nextPrincipalPayment': '#9165AE',
            'principalOutstanding': '#C2B2C8',
            'badDebtFundsReceived': '#FF8000',
            'nextBadDebtFundsPayment': '#FAAC58',
            'badDebtFundsOutstanding': '#F5D0A9',
            'profitsReceived': '#72C02C', 
            'nextProfitsPayment': '#a2de6e',
            'profitsOutstanding': '#d7f1c1'
        }
        
        names = {
            'feesPaid': 'Fees Paid', 
            'nextFeesPayment': 'Next Fee Payment', 
            'feesOutstanding': 'Fees Outstanding', 
            'principalReceived': 'Principal Received',
            'nextPrincipalPayment': 'Next Principal Payment',
            'principalOutstanding': 'Principal Outstanding', 
            'badDebtFundsReceived': 'Bad Debt Funds Received',
            'nextBadDebtFundsPayment': 'Next Bad Debt Funds Payment',
            'badDebtFundsOutstanding': 'Bad Debt Funds Outstanding',
            'profitsReceived': 'Profits Received', 
            'nextProfitsPayment': 'Next Profit Payment',
            'profitsOutstanding': 'Profits Outstanding'
        }
        
        groups = {
            'feesPaid': 'Fees', 
            'nextFeesPayment': 'Fees', 
            'feesOutstanding': 'Fees',  
            'principalReceived': 'Principal',
            'nextPrincipalPayment': 'Principal',
            'principalOutstanding': 'Principal', 
            'badDebtFundsReceived': 'Bad Debt Funds',
            'nextBadDebtFundsPayment': 'Bad Debt Funds',
            'badDebtFundsOutstanding': 'Bad Debt Funds',
            'profitsReceived': 'Profits', 
            'nextProfitsPayment': 'Profits',
            'profitsOutstanding': 'Profits'
        }
        
        return go.Bar(
            x = _convert_to_list(colName),
            y = companiesList,
            name = names[colName],
            orientation = 'h',
            width = 0.75,
            hoverinfo = 'text',
            text = ['' if val == 0 else '${}'.format(val) for val in _convert_to_list(colName)],
            legendgroup = groups[colName],
            marker = {
                'color': colourDict[colName],
                'line': {'width': 0.}
            }
        )
    
    # Define helper function
    def _min_max(x, minVal, maxVal):
        assert minVal <= maxVal
        return max(minVal, min(x, maxVal))
    
    def _capitalise_first_letter(string):
        return ''.join([char.upper() if i == 0 else char for i, char in enumerate(string)])
    
    # Define udfs
    @F.udf(returnType=T.FloatType())
    def calc_received(val, maxVal, *args):
        return _min_max(val - sum(args), 0., maxVal)
    
    @F.udf(returnType=T.FloatType())
    def calc_next_payment(val, maxVal, unitPayment, *args):
        # Handle case when nextPMT spills over from previous category
        if (val - sum(args) < 0):
            return _min_max(unitPayment - (sum(args) - val), 0., unitPayment)
        
        remainder = _min_max(maxVal - (val - sum(args)), 0., maxVal)
        return min(remainder, unitPayment)
    
    @F.udf(returnType=T.FloatType())
    def calc_outstanding(val, maxVal, unitPayment, *args):
        return _min_max(maxVal - (val + unitPayment - sum(args)), 0., maxVal)
    
    @F.udf(returnType=T.IntegerType())
    def months_to_breakeven(nextPrincipalPayment, principalOutstanding, unitPayment):
        return int(math.ceil((nextPrincipalPayment + principalOutstanding) / unitPayment))
    
        
    
    # Add relevant columns to notesDF
    
    categories = ['fees', 'principal', 'badDebtFunds', 'profits']
    
    # Add `received`, `next_payment` and `outstanding` columns programmatically
    for i, category in enumerate(categories):
        DF = (DF
              .withColumn('{}Received'.format(category) if category != 'fees' else 'feesPaid', 
                          calc_received('amountRepayed', category, *categories[:i]))
              .withColumn('next{}Payment'.format(_capitalise_first_letter(category)), 
                          calc_next_payment('amountRepayed', category, 'unitPayment', *categories[:i]))
              .withColumn('{}Outstanding'.format(category), 
                          calc_outstanding('amountRepayed', category, 'unitPayment', *categories[:i])))
        
    # Add `monthsToBreakeven` column
    DF = (DF.withColumn('monthsToBreakeven', 
                        months_to_breakeven('nextPrincipalPayment',
                                            'principalOutstanding',
                                            'unitPayment'))
          .orderBy(['feesPaid', 'principal', 'principalOutstanding', 'interestRate', 'cyclesTotal'], 
                   ascending=[1, 1, 0, 1, 1])
          .cache())
    
    # Obtain list of companies
    companiesList = _convert_to_list('company')
    
    # Create keys for traces
    _megaList = [['{}Received'.format(category) if category != 'fees' else 'feesPaid', 
                 'next{}Payment'.format((_capitalise_first_letter(category))), 
                 '{}Outstanding'.format(category)] for category in categories]
    
    keys = [item for sublist in _megaList for item in sublist]
    
    # Create traces
    traces = []
    for key in keys:
        traces.append(_create_trace(key))
    data = go.Data(traces)
    
    # Create annotations
    annotationData = (DF
                      .select('company', 'principal', 'monthsToBreakeven')
                      .collect())
    annotations = [{
            'x': row['principal'] - 2,
            'y': row['company'],
            'text': row['monthsToBreakeven'] if row['monthsToBreakeven'] > 0 else '',
            'font': {
                'family': 'Arial',
                'size': 14,
                'color': 'rgb(255, 255, 255)'},
            'showarrow': False
        } for row in annotationData]
    
    # Create layout
    layout = go.Layout(
        height = 30 * notesDF.count() + 280,
        xaxis = {
            'domain': [0.2, 1],
            'tickprefix': '$'
        },
        barmode = 'stack',
        legend = {
            'orientation': 'h',
            'traceorder': 'grouped'
        },
        annotations = annotations
    )
    
    figure = go.Figure(data=data, layout=layout)
    
    pyo.iplot(figure)

In [15]:
progress_bar(notesDF)

In [105]:
fundsDict = notesDF.groupBy('grade').agg(F.sum('principal')).rdd.collectAsMap()

In [106]:
fundsDict('A')

TypeError: 'dict' object is not callable

In [109]:
def wind_rose(DF):
    '''
    Visualise principal invested in notes by grades.
    '''
    
    grades = ['A+', 'A', 'B+', 'B', 'C+', 'C', 'D+', 'D', 'E+', 'E']
    fundsDict = (DF
                 .groupBy('grade')
                 .agg(F.sum('principal'))
                 .rdd
                 .collectAsMap())
    
    trace = go.Area(
        r = [fundsDict[grade] if grade in fundsDict.keys() else 0 for grade in grades],
        t = grades,
        marker = {'color': '#9165AE'},
    )
    
    data = go.Data([trace])
    
    layout = go.Layout(
        orientation = -90,
        angularaxis = {
            'showticklabels': False,
            'showline': False,
        },
        radialaxis = {
            'orientation': -90,
            'visible': False,
            'showline': False
        },
        showlegend = False,
    )
    
    pyo.iplot(go.Figure(data=data, layout=layout))
    
wind_rose(notesDF)

In [114]:
def notes_distribution_bar_chart(DF):
    '''
    Plot a bar chart showing the fund distribution in each note grade.
    '''
    grades = ['A+', 'A', 'B+', 'B', 'C+', 'C', 'D+', 'D', 'E+', 'E']
    fundsDict = (DF
                 .groupBy('grade')
                 .agg(F.sum('principal'))
                 .rdd
                 .collectAsMap())
    
    trace = go.Bar(
        x = grades,
        y = [fundsDict[grade] if grade in fundsDict.keys() else 0 for grade in grades],
        marker = {'color': '#9165AE'},
    )
    
    data = go.Data([trace])
    
    layout = go.Layout(
        yaxis = {'tickprefix': '$'}
    )
    
    pyo.iplot(go.Figure(data=data, layout=layout))

notes_distribution_bar_chart(notesDF)

In [91]:
go.Area().help('customdata')

Current path: []
Current parent object_names: []

With the current parents, 'customdata' can be used as follows:

Under ('figure', 'data', 'area'):

    description: Assigns extra data each datum. This may be useful when listening to hover,
        click and selection events. Note that, *scatter* traces also appends
        customdata items in the markers DOM elements
    role: data





### TODO
- Design dashboard
    - Vertical bar chart showing distribution of funds
    - Change between different time periods
    - Wind rose showing distributino of funds for different grades
- Predictor bar chart showing expected returns if invest in new funds

In [16]:
testDF = spark.createDataFrame([(6.0, 'My Test Company', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.),
                                (20.0, 'My Test Company1', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.),
                                (70.0, 'My Test Company2', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.),
                                (80.0, 'My Test Company3', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.),
                                (115.0, 'My Test Company4', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.)],
                               ['amountRepayed', 'company', 'feesAccrued', 'interestRate', 
                                'principal', 'unitPayment', 'cyclesTotal', 'profits', 'badDebtFunds'])

display_DF(testDF)

,amountRepayed,company,feesAccrued,interestRate,principal,unitPayment,cyclesTotal,profits,badDebtFunds
0,6.0,My Test Company,3.0,16.18,75.0,10.0,36,18.14,5.0
1,20.0,My Test Company1,3.0,16.18,75.0,10.0,36,18.14,5.0
2,70.0,My Test Company2,3.0,16.18,75.0,10.0,36,18.14,5.0
3,80.0,My Test Company3,3.0,16.18,75.0,10.0,36,18.14,5.0
4,115.0,My Test Company4,3.0,16.18,75.0,10.0,36,18.14,5.0


In [17]:
@F.udf(returnType=T.FloatType())
    def bad_debt_funds_outstanding(amountRepayedNetFees, principal, badDebtFunds, nextBadDebtFundsPayment, unitPayment):
        return min(badDebtFunds - max(amountRepayedNetFees - principal - nextBadDebtFundsPayment - unitPayment, 0.), 0.)
        
        badDebt = min(amountRepayedNetFees - principal, badDebtFunds)
        
        if (badDebtFunds - badDebtFundsReceived) < unitPayment:
            return 0.0
        elif badDebtFundsReceived >= - unitPayment:
            return badDebtFunds - (badDebtFundsReceived + unitPayment)
        else:
            return badDebtFunds

IndentationError: unexpected indent (<ipython-input-17-4638c76ee0d9>, line 2)

In [ ]:
def _min_max(x, minVal, maxVal):
    return max(minVal, min(x, maxVal))

In [ ]:
progress_bar(testDF)